In [ ]:
import requests
import json
import pandas as pd
from getpass import getpass
from dateutil import parser
from datetime import timedelta

In [ ]:
# input username and password to get a token from the historian
def getViewerToken(url):
    # requesting user and pass
    print('Enter your username: ')
    user = input()
    print('\nEnter your password: ')
    pswd = getpass()
    # getting a token
    payload = """
                {\r\n  \"username\":\"%s\",
                \r\n  \"password\":\"%s\"\r\n}"
                """%(user,pswd)
    headers = {'Content-Type': 'application/json','Content-Type': 'application/json'}
    resp = requests.request("POST", url+'getUserToken', headers=headers, data=payload).text.encode('utf8')
    userToken = str(resp).split(r'"')[-2]
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        print('\nYour token is: %s'%userToken)
        return userToken
    else:
        print('The response was not Good, it looks like this: \n\n%s'%parsed)

# finding tags using search string
def searchTags(url,token,path,deep,search):
    payload = "{\r\n  \"userToken\": \"%s\",\r\n  \"path\": \"%s\",\r\n  \"deep\": %s,\r\n  \"search\": \"%s\"\r\n}"%(token,path,deep,search)
    headers = {'Content-Type': 'application/json','Content-Type': 'text/plain'}
    resp = requests.request("POST", url+'browseTags', headers=headers, data=payload).text
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        return parsed['tags']
    else:
        print('The response was not Good, it looks like this: \n\n%s'%parsed)

# getting a dataframe of the raw data from the tags for the specific timeframe
def getTagDataFrame(url,token,tags,startTime,endTime):
    payload = "{\r\n  \"userToken\": \"%s\",\r\n  \"tags\": %s,\r\n  \"startTime\": \"%s\",\r\n  \"endTime\": \"%s\"\r\n}"%(token,tags,startTime,endTime)
    headers = {'Content-Type': 'application/json','Content-Type': 'text/plain'}
    resp = requests.request("POST", url+'getTagData', headers=headers, data = payload).text
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        cols = ['full_branch','tag','t','v']
        df_full = pd.DataFrame(columns=cols)
        data_dict = parsed['data']
        for key in data_dict:
            df_key = pd.DataFrame(data_dict[key])
            df_key.insert(0, 'tag', key.split('.')[-1])
            df_key.insert(0, 'full_branch', key)
            df_full = pd.concat([df_full,df_key])
        df_full = df_full.reset_index(drop=True)
        return df_full
    else:
        print('The response was not Good, it looks like this: \n\n%s'%parsed)
        
# getting inputs from user for searching tags and outputting tag data to a dataframe
def searchTagsAndGetDataframe():
    url = "http://hostname:port/api/v2/"
    token = getViewerToken(url)
    # requesting a path
    print('\nEnter a path in the historian to search through, or leave blank: ')
    path = input()
    # checking for recursive (deep) searching
    print('\nWould you like to recursively search through all branches in the path? ([Y]/n)')
    deep_resp = input()
    chk = 0
    while chk == 0:
        if deep_resp.lower() == 'y':
            deep = 'true'
            chk = 1
        elif deep_resp.lower() == 'n':
            deep = 'false'
            chk = 1
        elif deep_resp == '':
            deep = 'true'
            chk = 1
        else:
            print('\nPlease use "y" or "n" to indicate whether you want to search through all branches in the path.')
            deep_resp = input()
    # getting a search string
    print('\nEnter a search string to find the tags you want data from: ')
    search_resp = input()
    chk2 = 0
    while chk2 == 0:
        if search_resp == '':
            print('\nPlease enter a search string: ')
            search_resp = input()
        else:
            search = search_resp
            tags = searchTags(url,token,path,deep,search)
            for t in tags:
                print(t)
            print('\n\nWould you like to get the data for this set of tags? (y/[N])')
            tagchk_resp = input()
            if tagchk_resp.lower() == 'y':
                chk2 = 1
            else:
                print('\nEnter a search string to find the tags you want data from: ')
                search_resp = input()
    # getting start and end date / times
    print('\nWhat is the start date / time? Ex. "now-7d" means the start date will be a week ago.')
    startTime = input()
    print('\nWhat is the end date / time? Default = "now"')
    endTime_resp = input()
    if deep_resp.lower() == '':
        endTime = 'now'
    else:
        endTime = endTime_resp
    #getting the dataframe, writing it to Excel and returning it
    df = getTagDataFrame(url,token,tags,startTime,endTime)
    df['t'] = pd.to_datetime(df['t'])
    return df

In [ ]:
df_upchkpre = searchTagsAndGetDataframe()

In [ ]:
df_upchkpost = searchTagsAndGetDataframe()

In [ ]:
df_upchkpre.head(10)

In [ ]:
df_upchkmerge = pd.merge(df_upchkpre, df_upchkpost, on=['t'])

In [ ]:
df_upchkmerge['val_chk'] = df_upchkmerge['v_x'].equals(df_upchkmerge['v_y'])

In [ ]:
df_upchkmerge['val_chk'].unique()